In [4]:
import json
import numpy as np
from fbpathtrain import VectorizedData
import random
import re
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
import sys
import time
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
import sys
from sklearn import cross_validation

from __future__ import print_function

In [6]:
def q_to_fdict(q):
    fdict = {}
    for lat in q['LAT']:
        if (lat['type'] != "WordnetLAT"):
            fdict['lat/' + lat['text'] + '/' + lat['type']] = 1
    for sv in q['SV']:
        fdict['sv'] = sv
    if (len(q['SV']) == 0):
        fdict['sv_not_present'] = 1
#     print (fdict)
    return fdict

In [42]:
with open('data/train-data.json', 'r') as f:
    fdict = [q_to_fdict(q) for q in json.load(f)]
    Xdict = DictVectorizer()
    trainX = Xdict.fit_transform(fdict)
    
with open('data/test-data.json', 'r') as f:
    fdict = [q_to_fdict(q) for q in json.load(f)]
    testX = Xdict.transform(fdict)
   

In [46]:
with open('data/train-data.tsv', 'r') as f:
    trainY = [line.split("\t")[3].replace("\n","") for line in f]

with open('data/test-data.tsv', 'r') as f:
    testY = [line.split("\t")[3].replace("\n","") for line in f]


cfier = LogisticRegression(solver='lbfgs', multi_class='multinomial')
cfier.fit(trainX, trainY)
print (cfier.score(trainX, trainY))

#Temporary solution: cross validation
res = cross_validation.cross_val_score(cfier, trainX, trainY, cv=10)
print ("10 fold cross-validation accuracy:")
print (res)
print ("Average over folds")
print (sum(res) / float(len(res)))
print ("Accuracy on test data set")
print (cfier.score(testX, testY))
cfier.coef_.tolist

0.933418928833
10 fold cross-validation accuracy:
[ 0.81569343  0.77919708  0.80804388  0.79304029  0.80586081  0.76330275
  0.78637201  0.79189687  0.79005525  0.79373849]
Average over folds
0.792720085293
Accuracy on test data set
0.864


<function tolist>

In [33]:
#Error analysis
testX[0]
s = set()
[s.add(e) for e in trainY]
print (sorted(s))
with open('data/test-data.tsv', 'r') as f:
    for i, line in enumerate(f):
        if (cfier.predict(testX[i]) != testY[i]):
            splitted = line.replace("\n", "").split("\t")
            idxs = []
            for key in fdict[i]:
                if (key == 'sv'):
                    key = key + "=" + fdict[i][key]
                idxs.append(Xdict.feature_names_.index(key) if key in Xdict.feature_names_ else 'Not present')
            coefs = [[c[idx] for c in cfier.coef_] if idx != 'Not present' else 'Not present' for idx in idxs]
            print (splitted[0], splitted[2], splitted[3], cfier.predict(testX[i]), fdict[i])
            print ("Class coeficients for LATs and SV:" + str(coefs))
            print ()
            

['ABBR:abb', 'ABBR:exp', 'DESC:def', 'DESC:desc', 'DESC:manner', 'DESC:reason', 'ENTY:animal', 'ENTY:body', 'ENTY:color', 'ENTY:cremat', 'ENTY:currency', 'ENTY:dismed', 'ENTY:event', 'ENTY:food', 'ENTY:instru', 'ENTY:lang', 'ENTY:letter', 'ENTY:other', 'ENTY:plant', 'ENTY:product', 'ENTY:religion', 'ENTY:sport', 'ENTY:substance', 'ENTY:symbol', 'ENTY:techmeth', 'ENTY:termeq', 'ENTY:veh', 'ENTY:word', 'HUM:desc', 'HUM:gr', 'HUM:ind', 'HUM:title', 'LOC:city', 'LOC:country', 'LOC:mount', 'LOC:other', 'LOC:state', 'NUM:code', 'NUM:count', 'NUM:date', 'NUM:dist', 'NUM:money', 'NUM:ord', 'NUM:other', 'NUM:perc', 'NUM:period', 'NUM:speed', 'NUM:temp', 'NUM:volsize', 'NUM:weight']
1 What county is Modesto , California in ? LOC ['LOC:other'] {'sv_not_present': 1, u'lat/county/LAT': 1}
Class coeficients for LATs and SV:[[0.8233595878689226, 0.46641381403342919, 2.1502095225097295, 0.084541478537811368, -1.8063403005445271, -0.66825883488566296, 0.073668161165201348, -0.57732743661598562, 0.00485

In [34]:
with open('data/yoda-questions.json', 'r') as f:
    fdict = [q_to_fdict(q) for q in json.load(f)]
    testX = Xdict.transform(fdict)
    
with open('data/yoda-questions.tsv', 'r') as f:
    for i, line in enumerate(f):
        print (i, line.replace("\n", ""), cfier.predict(testX[i]), fdict[i])

0 1756	factoid	When is Fashion week in NYC?	answer ['NUM:date'] {u'lat/date/QuestionWordLAT': 1, 'sv_not_present': 1, u'lat/time/QuestionWordLAT': 1}
1 1434	factoid	What site did Lindbergh begin his flight from in 1927?	answer ['LOC:other'] {u'lat/site/LAT': 1, 'sv': u'begin'}
2 1790	factoid	What country is the holy city of Mecca located in?	answer ['LOC:country'] {u'lat/country/LAT': 1, 'sv': u'located'}
3 1992	factoid	How hot is the sun?	answer ['DESC:def'] {u'lat/hot/LAT': 1, 'sv_not_present': 1}
4 2128	factoid	What flavor filling did the original Twinkies have?	answer ['DESC:def'] {u'lat/filling/LAT': 1, 'sv_not_present': 1}
5 1840	factoid	What is the state song of Kansas?	answer ['ENTY:cremat'] {'sv_not_present': 1, u'lat/song/LAT': 1}
6 1689	factoid	What war is connected with the book "Charge of the Light Brigade"?	answer ['ENTY:event'] {u'lat/war/LAT': 1, 'sv': u'connected'}
7 10040	factoid	Where was the first atomic bomb detonated?	answer ['LOC:other'] {u'lat/location/QuestionW

In [106]:
import math
idx = 27
cls = 4
print (cfier.predict(testX[idx]))
pred = cfier.predict_proba(testX[idx])[0]
print (sum(pred))
print (pred)
res = [1.0 / (1.0 + math.exp(-cfier.intercept_[cls] - testX.getrow(idx).dot(cfier.coef_[cls]))) for cls in range(6)] 
print (res)
print ([r/sum(res) for r in res])


['NUM']
1.0
[ 0.02565605  0.12500891  0.21361431  0.16216415  0.0873067   0.38624988]
[0.06587366247997618, 0.3209689360896998, 0.548469356659683, 0.4163675485305018, 0.22416591393764587, 0.9917229946397051]
[0.025656049577277885, 0.12500891292611263, 0.21361431073239487, 0.1621641497573534, 0.0873066956504546, 0.3862498813564065]


In [1]:
import gensim

word_vector_path = "data/glove.6B.50d.txt"
word_vector = gensim.models.Word2Vec.load_word2vec_format(word_vector_path, binary=False)

In [2]:
def create201dim_vectors(json_filename, tsv_filename):
    with open(json_filename, 'r') as f:
        json_dict = json.load(f)

    res = []
    with open(tsv_filename, 'r') as f:
        for i, line in enumerate(f):
            question = line.split("\t")[2].lower()
            try:
                first_word_vec = word_vector[question.split(" ")[0]]
            except KeyError:
                first_word_vec = np.zeros(50)
            try:
                second_word_vec = word_vector[question.split(" ")[1]]
            except KeyError:
                second_word_vec = np.zeros(50)
            lat_array = [lat['text'] for lat in json_dict[i]['LAT'] if lat['type'] != "WordnetLAT"]
            sv_array = json_dict[i]['SV']
            lat_vec = np.zeros(50)
            for w in lat_array:
                try:
                    tmp = word_vector[w.lower()]
                except:
                    tmp = np.zeros(50)
                lat_vec = lat_vec + tmp
            if (len(lat_array) != 0):
                lat_vec = lat_vec / float(len(lat_array))
            else:
                lat_vec = np.zeros(50)
#             lat_vec = np.mean([word_vector[w.lower()] if w.lower() in word_vector else np.zeros(50) for w in lat_array], axis=0)
#             print lat_vec
#             if (len(lat_array) == 1 and lat_array[0].lower() in word_vector):
#                 lat_vec = word_vector[lat_array[0].lower()]
#             else:
#                 lat_vec = np.zeros(50)
            if (len(sv_array) == 1 and sv_array[0].lower() in word_vector):
                sv_vec = word_vector[sv_array[0].lower()]
                flag = 0
            else:
                sv_vec = np.zeros(50)
                flag = 1
            vec = np.concatenate((first_word_vec, second_word_vec, lat_vec, sv_vec, np.zeros(1) + flag))
            res.append(vec)
    return res

In [47]:
#201 dimensional vectors; vectors of first two words from question plus LAT and SV


 
train200vectors = create201dim_vectors('data/train-data.json', 'data/train-data.tsv')

cfier = LogisticRegression(solver='lbfgs', multi_class='multinomial')
cfier.fit(train200vectors, trainY)
 
test200vectors = create201dim_vectors('data/test-data.json', 'data/test-data.tsv')
        
cfier.score(test200vectors, testY)

print (len(train200vectors))
print (len(test200vectors))
# print (testX)
from scipy.sparse import hstack
mergedTrain = hstack((train200vectors, trainX))
mergedTest = hstack((test200vectors, testX))


cfier = LogisticRegression(solver='lbfgs', multi_class='multinomial')
cfier.fit(mergedTrain, trainY)
        
print (cross_validation.cross_val_score(cfier, mergedTrain, trainY, cv=10))    
print (cfier.score(mergedTest, testY))
testX

5452
500
[ 0.89233577  0.84854015  0.86837294  0.88095238  0.9029304   0.84954128
  0.86003683  0.86924494  0.86740331  0.85267035]
0.898


<500x2837 sparse matrix of type '<type 'numpy.float64'>'
	with 835 stored elements in Compressed Sparse Row format>

In [31]:
#Error analysis

#TODO: Table of weights of features, most similar words to segments of weight vector 
with open('data/test-data.json', 'r') as f:
    json_dict = json.load(f)
    
with open('data/test-data.tsv', 'r') as f:
    for i, line in enumerate(f):
        if (cfier.predict(mergedTest.getrow(i)) != testY[i]):
            print (line.replace("\n", ""), cfier.predict(mergedTest.getrow(i)), json_dict[i]['SV'], [lat['text'] for lat in json_dict[i]['LAT'] if lat['type'] != "WordnetLAT"])

27	LOC:other	What imaginary line is halfway between the North and South Poles ?	LOC ['ENTY'] [] [u'line']
44	ENTY:animal	What is the proper name for a female walrus ?	ENTY ['DESC'] [] []
59	NUM:speed	What is the speed hummingbirds fly ?	NUM ['ENTY'] [] [u'fly']
61	HUM:ind	What was W.C. Fields ' real name ?	HUM ['ENTY'] [] [u'fields', u'field']
75	DESC:def	What is the pH scale ?	DESC ['ENTY'] [] [u'scale']
99	ENTY:other	What is the longest major league baseball-winning streak ?	ENTY ['NUM'] [u'baseball-winning'] [u'streak']
106	NUM:temp	The sun 's core , what is the temperature ?	NUM ['ENTY'] [] [u'core']
117	ENTY:substance	What mineral helps prevent osteoporosis ?	ENTY ['LOC'] [u'helps'] [u'mineral']
128	DESC:def	What is Wimbledon ?	DESC ['LOC'] [] [u'location']
135	ENTY:other	What is the major fault line near Kentucky ?	ENTY ['DESC'] [] [u'line']
140	DESC:def	What is severance pay ?	DESC ['NUM'] [] [u'pay']
148	DESC:def	What does Phi Beta Kappa mean ?	DESC ['ABBR'] [u'mean'] [u'organi

In [136]:
idx = 0
s = set()
[s.add(e) for e in trainY]
s = sorted(s)
# print (cfier.predict(test200vectors[idx]))
# print (cfier.predict_proba(test200vectors[idx]))

for i, weight_vector in enumerate(cfier.coef_):
    print ("Class name: " + s[i])
    print ("First word:")
    print (word_vector.most_similar(positive=[weight_vector[:50]]))
    print ("Second word:")
    print (word_vector.most_similar(positive=[weight_vector[50:100]]))
    print ("Support verb:")
    print (word_vector.most_similar(positive=[weight_vector[100:150]]))
    print ("LAT:")
    print (word_vector.most_similar(positive=[weight_vector[150:200]]))
    print ("SV not present flag:")
    print (weight_vector[200:201])

Class name: ABBR
First word:
[(u'moneyline', 0.7154236435890198), (u'qap', 0.7011500597000122), (u'noisecreep', 0.6815968751907349), (u'wakamiya', 0.6767191886901855), (u'belorusskaya', 0.6736595630645752), (u'al-arabiya', 0.6730930209159851), (u'newstalkzb', 0.6696533560752869), (u'kacl', 0.669644296169281), (u'b.b.c.', 0.669158935546875), (u'wtcg', 0.6644538640975952)]
Second word:
[(u'thinks', 0.7000970840454102), (u'does', 0.6979345083236694), (u'asks', 0.6614990234375), (u'agrees', 0.6588461995124817), (u'responds', 0.6519638895988464), (u'understands', 0.6457714438438416), (u'decides', 0.6331676840782166), (u'assures', 0.6288090944290161), (u'pretends', 0.6233050227165222), (u'gives', 0.6220813989639282)]
Support verb:
[(u'kvb', 0.7588256001472473), (u'prm', 0.734941303730011), (u'uac', 0.7319374084472656), (u'bcu', 0.7307842373847961), (u'cnp', 0.7290647029876709), (u'mjc', 0.723932147026062), (u'ncec', 0.7226738333702087), (u'atu', 0.7218532562255859), (u'asf', 0.72002375125885